#SECTION 0: Mount Drive

In [153]:
#@title Generar Token de Acceso a Carpetas Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [154]:
#@title Listar carpetas en PRI_CNIP_ESTRUCTURAS/computos/edo/casillas/*mun*
import pandas as pd
from IPython.display import display, HTML


dir_computos="/gdrive/My\ Drive/PRI_CNIP_ESTRUCTURAS/computos/"
dir_mun_pattern_sufix='/'+'*casillas*/' + '*municipio*'

def get_list_ls(pattern):
  'return list from ls pattern and its len'
  list_ls=[]
  try: 
    list_ls = !ls {pattern} | tr '\n' '\n'
  except:
    pass
  if 'cannot' in list_ls[0]:
    list_ls = []
  return list_ls, len(list_ls)


def found_links_files(dir_computos,dir_mun_pattern_sufix):
  ''' Return links to files with pattern ''' 
  list_boxes_in_edo=[]
  list_data_edo=[]
  boxes_in_edo=[]

  list_edos, _total = get_list_ls(dir_computos)
  
  list_edos= filter(lambda x: '0_mexico' not in x,list_edos)
  
  for edo in list_edos:

    sub_dir = dir_computos + edo + dir_mun_pattern_sufix
    boxes_in_edo,total_box_files= get_list_ls(pattern=sub_dir)

    list_boxes_in_edo.extend(boxes_in_edo)
    list_data_edo.append({'edo': edo, 
                          'found_files':total_box_files })
    if not total_box_files:
      print(f'El estado {edo} no tiene el patrón casillas municipios ')

  df_total_files=pd.DataFrame(list_data_edo)
  df_total_files=df_total_files.rename(columns={'edo':'Carpeta Estado', 'found_files': 'Total Found Files'})
  df_box=pd.DataFrame(list_boxes_in_edo, columns=['ruta'])

  return df_total_files,df_box

df_total_files,df_box = found_links_files(dir_computos,dir_mun_pattern_sufix )


display(HTML(df_total_files.sort_values(by='Total Found Files', ascending=False).to_html()))
display(HTML(df_box.to_html()))


El estado 10_durango no tiene el patrón casillas municipios 
El estado 17_morelos no tiene el patrón casillas municipios 


,Carpeta Estado,Total Found Files
3,13_hidalgo,4
4,14_jalisco,4
16,25_sinaloa,3
14,23_quintana-roo,3
30,8_chihuahua,3
28,6_colima,3
27,5_coahuila,3
26,4_campeche,3
25,3_baja-california-sur,3
21,2_baja-california,3


,ruta
0,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/11_guanajuato/guanajuato_casillas/guanajuato_municipios_2012_polling_place.xlsx
1,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/11_guanajuato/guanajuato_casillas/guanajuato_municipios_2015_polling_place.xlsx
2,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/11_guanajuato/guanajuato_casillas/guanajuato_municipios_2018_polling_place.xlsx
3,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/12_guerrero/12_guerrero_casillas/guerrero_municipios_2015_polling_place.xlsx
4,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/12_guerrero/12_guerrero_casillas/guerrero_municipios_2018_polling_place.xlsx
5,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2008_polling_place.xlsx
6,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2011_polling_place.xlsx
7,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2016_polling_place.xlsx
8,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2017_polling_place.xlsx
9,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/14_jalisco/jalisco_casillas/jalisco_municipios_2012_polling_place (1).xlsx


-------

# SECTION 1:  COLUMN CONTINUITY

In [158]:
#@title functions [continuity]
import pandas as pd

def continuidad_columna(content_books=[], 
                        sheet_name='Polling_places_corrections',
                        column_to_evaluate='local_district_id'
                        ):
  ''' Evaluá si existe continuidad en columma determinada de una hoja del libro dada, compromando que sea vacía la diferencia simétrica'''
  evaluacion_distritos={}

  #print ('\nPara todos los archivos siguientes se evaluará la hoja:',sheet_name,'y la columna:' ,column_to_evaluate)
  for book in content_books:

    #print('\nLibro:',book)
    try:
      df_temp = pd.read_excel(book,sheet_name=sheet_name)
      temp_columns=list(df_temp.columns.values)
      temp_types=set(df_temp.dtypes.values)
      temp_types_by_column=dict(df_temp.dtypes)


      
    except:
      print(f'\tNo existe la hoja {sheet_name} en el archivo: {book}')
      continue

    try:
      book = book.split('/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/')[1]
    except:
      book=book

    filter_independents=list(filter(lambda x: 'dependiente' in x, temp_columns))

    
    evaluacion_distritos[book]={}

    _df=df_temp.agg({column_to_evaluate:['max','min']})
    _unicos = df_temp[column_to_evaluate].unique()

    evaluacion_distritos[book]['min'] = _df[column_to_evaluate]['min']
    evaluacion_distritos[book]['max'] = _df[column_to_evaluate]['max']
    evaluacion_distritos[book]['restaMaxMin'] = _df[column_to_evaluate]['max']-_df[column_to_evaluate]['min']
    evaluacion_distritos[book]['shape']= df_temp.shape
    evaluacion_distritos[book]['size']= df_temp.size
    evaluacion_distritos[book]['columns']=temp_columns
    evaluacion_distritos[book]['types']=temp_types
    evaluacion_distritos[book]['filter_independents']=filter_independents
    evaluacion_distritos[book]['total_independents']=len(filter_independents)

    evaluacion_distritos[book]['temp_types_by_column']=temp_types_by_column

          

    if len(temp_types) == 1:
      evaluacion_distritos[book]['temp_types_by_column']='Only One Type'



      
    if not filter_independents:
      evaluacion_distritos[book]['filter_independents']='Sin independientes'




    if not evaluacion_distritos[book]['restaMaxMin']:
      evaluacion_distritos[book]['restaMaxMin']=-1
    else:
      evaluacion_distritos[book]['restaMaxMin']=len(set(_unicos))



    a = set(_unicos)
    b = set(np.arange(evaluacion_distritos[book]['min'],evaluacion_distritos[book]['max'] + 1))
    c = a^b
    evaluacion_distritos[book]['total_diferencia']=len(c)
    evaluacion_distritos[book]['diferencia']=", ".join(map(str,c))

  return evaluacion_distritos

In [159]:
#@title Process to evaluate continuity in a column
import numpy as np
from IPython.display import display, HTML
#display(HTML(filter_df_continuity.to_html()))

#'''content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
#content_books =  !ls {content_books}'''


content_books=list(df_box['ruta'])
#content_books=['/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx']


#content_books=['san-luis-potosi_2018_polling_place (3) (1).xlsx']
column_to_evaluate =  "municipality_id"#@param {type:"string"}
sheet_name="Polling_places"#@param {type:"string"}

truncate_discontinuous_ids = True #@param {type:"boolean"}


evaluacion_distritos=continuidad_columna(content_books=content_books, 
                        sheet_name=sheet_name,
                        column_to_evaluate=column_to_evaluate
                        )

df = pd.DataFrame([[key.split('.')[0],value['total_diferencia'],value['diferencia'],
                      value['restaMaxMin'],value['min'], value['max'], value['shape'], value['size'], 
                      value['columns'], value['filter_independents'], value['total_independents'],  value['types'], value['temp_types_by_column']] for key,value in evaluacion_distritos.items()], 
                  columns=['Libro', 'Total_No_continuos','Lista IDs NO Continuos', 'Total Ids únicos','Mínimo', 'Máximo', 'Filas,Columnas', 'Total Celdas', 'columns', 'filter_independents','total_independents','types', 'temp_types_by_column'])

if truncate_discontinuous_ids:
  df = pd.DataFrame([[key.split('.')[0],value['total_diferencia'],value['diferencia'][0:50],
                      value['restaMaxMin'],value['min'], value['max'], value['shape'], value['size'], 
                      value['columns'], value['filter_independents'], value['total_independents'],value['types'], value['temp_types_by_column']] for key,value in evaluacion_distritos.items()], 
                  columns=['Libro', 'Total_No_continuos','Lista IDs NO Continuos', 'Total Ids únicos','Mínimo', 'Máximo', 'Filas,Columnas', 'Total Celdas', 'columns', 'filter_independents','total_independents','types', 'temp_types_by_column'])

display(HTML(df.to_html()))
#df.to_excel('/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/data_from_notebook/continuity_data.xlsx')


	No existe la hoja Polling_places en el archivo: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/16_michoacan/16_michoacan_casillas/michoacan_municipios_2012_polling_place.xlsx
	No existe la hoja Polling_places en el archivo: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/16_michoacan/16_michoacan_casillas/michoacan_municipios_2015_municipality.xlsx
	No existe la hoja Polling_places en el archivo: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/24_san-luis-potosi/san-luis-potosi_casillas/san-luis-potosi_municipios_2012_polling_place.xlsx


,Libro,Total_No_continuos,Lista IDs NO Continuos,Total Ids únicos,Mínimo,Máximo,"Filas,Columnas",Total Celdas,columns,filter_independents,total_independents,types,temp_types_by_column
0,11_guanajuato/guanajuato_casillas/guanajuato_municipios_2012_polling_place,0,,46,1.0,46.0,"(6711, 15)",100665,"[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PT, PVEM, MC, PANAL, PAN,PANAL, PRI,PVEM, Not_reg, Null_votes]",Sin independientes,0,{int64},Only One Type
1,11_guanajuato/guanajuato_casillas/guanajuato_municipios_2015_polling_place,0,,46,1.0,46.0,"(7089, 21)",148869,"[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, PANAL, MORENA, PH, PES, PRI,PVEM, PRI,PANAL, PVEM,PANAL, PRI,PVEM,PANAL, C. Independiente 1, Not_reg, Null_votes]",[C. Independiente 1],1,{int64},Only One Type
2,11_guanajuato/guanajuato_casillas/guanajuato_municipios_2018_polling_place,0,,46,1.0,46.0,"(7463, 21)",156723,"[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, PANAL, MORENA, PES, MORENA,PT, MORENA,PES, PT,PES, C. Independiente 1, C. Independiente 2, C. Independiente 3, Not_reg, Null_votes]","[C. Independiente 1, C. Independiente 2, C. Independiente 3]",3,{int64},Only One Type
3,12_guerrero/12_guerrero_casillas/guerrero_municipios_2015_polling_place,0,,81,1.0,81.0,"(4806, 23)",110538,"[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PT, PVEM, MC, PANAL, MORENA, PH, PES, PPGRO, PRI,PVEM, PRD,PT, PRI,PVEM,PANAL, PRI,PANAL, PVEM,PANAL, C. Independiente 1, Not_reg, Null_votes]",[C. Independiente 1],1,{int64},Only One Type
4,12_guerrero/12_guerrero_casillas/guerrero_municipios_2018_polling_place,0,,81,1.0,81.0,"(9900, 33)",326700,"[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, PANAL, MORENA, PES, PPGRO, IHG, CG, PS, PSG, PAN,PRD,MC, PAN,PRD, PAN,MC, PRD,MC, PRI,PVEM, MORENA,PES, C. Independiente 1, C. Independiente 2, C. Independiente 3, C. Independiente 4, C. Independiente 5, C. Independiente 6, C. Independiente 7, Not_reg, Null_votes]","[C. Independiente 1, C. Independiente 2, C. Independiente 3, C. Independiente 4, C. Independiente 5, C. Independiente 6, C. Independiente 7]",7,{int64},Only One Type
5,13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2008_polling_place,0,,84,1.0,84.0,"(3219, 15)",48285,"[section_id, local_disctrict_id, municipality_id, state_id, PAN, PRI, PRD, PT, PVEM, MC, PSI, PANAL, PRI,PANAL, PRD,PT, Not_reg]",Sin independientes,0,{int64},Only One Type
6,13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2011_polling_place,0,,84,1.0,84.0,"(3432, 15)",51480,"[section_id, local_disctrict_id, municipality_id, state_id, PAN, PRI, PRD, PT, PVEM, MC, PANAL, PRD,PAN, PT,MC, PRI,PVEM,PANAL, Null_votes]",Sin independientes,0,{int64},Only One Type
7,13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2016_polling_place,1,45,83,1.0,84.0,"(3600, 22)",79200,"[section_id, local_disctrict_id, municipality_id, state_id, PAN, PRI, PRD, PT, PVEM, MC, PANAL, MORENA, PES, PRI,PVEM,PANAL, PRI,PVEM, PRI,PANAL, PVEM,PANAL, C. Independiente 1, C. Independiente 2, C. Independiente 3, Not_reg, Null_votes]","[C. Independiente 1, C. Independiente 2, C. Independiente 3]",3,{int64},Only One Type
8,13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2017_polling_place,0,,-1,45.0,45.0,"(14, 11)",154,"[section_id, local_disctrict_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, Null_votes]",Sin independientes,0,{int64},Only One Type
9,14_jalisco/jalisco_casillas/jalisco_municipios_2012_polling_place (1),0,,125,1.0,125.0,"(8892, 15)",133380,"[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PT, PVEM, MC, PANAL, PRI,PVEM, PT,MC, Not_reg, Null_votes]",Sin independientes,0,{int64},Only One Type


In [161]:
#@title save df to file

df = pd.DataFrame([[key.split('.')[0],value['total_diferencia'],value['diferencia'],
                      value['restaMaxMin'],value['min'], value['max'], value['shape'], value['size'], 
                      value['columns'], value['filter_independents'], value['total_independents'],  value['types'], value['temp_types_by_column']] for key,value in evaluacion_distritos.items()], 
                  columns=['Libro', 'Total_No_continuos','Lista IDs NO Continuos', 'Total Ids únicos','Mínimo', 'Máximo', 'Filas,Columnas', 'Total Celdas', 'columns', 'filter_independents','total_independents','types', 'temp_types_by_column'])

df.to_excel('/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/data_from_notebook/evaluate_continuity_column_municipality_all_rutes_mi otro nombre.xlsx')


In [162]:
df.columns

Index(['Libro', 'Total_No_continuos', 'Lista IDs NO Continuos',
       'Total Ids únicos', 'Mínimo', 'Máximo', 'Filas,Columnas',
       'Total Celdas', 'columns', 'filter_independents', 'total_independents',
       'types', 'temp_types_by_column'],
      dtype='object')

In [164]:
#@title graph df
variable_sort = "Total Celdas" #@param {type:"string"}

import plotly.express as px


fig = px.bar(df.sort_values(by=variable_sort, ascending=False), 
             y=variable_sort,
             x='Libro',
             hover_data=['Total Celdas','Filas,Columnas','Total_No_continuos' ,'Total Ids únicos', 'total_independents','Libro'], 
             color=variable_sort,
             labels={'Total Celdas':'Cells'},
  
             height=400)

fig.update_layout(title='View '+ variable_sort,
                  xaxis_title='Libro',
                  yaxis_title=variable_sort             
                  )

fig.update_xaxes(tickangle=90, tickfont=dict(family='arial', color='gray', size=6))



fig.show()


## SUBSECTION FILTER SUMMARY CONTINUIDAD

In [165]:
#@title Ver Total_No_continuos > 0
df_continuity=df.copy()
filter_df_continuity=df_continuity[ df_continuity['Total_No_continuos']>0]
estados=list(map(lambda x: x.split('/')[0], filter_df_continuity['Libro']))
filter_df_continuity['Lista IDs NO Continuos']=list(map(lambda x: x[:25], filter_df_continuity['Lista IDs NO Continuos']))
filter_df_continuity['columns']=list(map(lambda x: x[4:10], filter_df_continuity['columns']))
filter_df_continuity['filter_independents']=list(map(lambda x: x[-2:-1], filter_df_continuity['filter_independents']))


filter_df_continuity.insert(0, "Estado", estados )
display(HTML(filter_df_continuity.to_html()))
print(list(filter_df_continuity['Libro']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,Estado,Libro,Total_No_continuos,Lista IDs NO Continuos,Total Ids únicos,Mínimo,Máximo,"Filas,Columnas",Total Celdas,columns,filter_independents,total_independents,types,temp_types_by_column
7,13_hidalgo,13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2016_polling_place,1,45,83,1.0,84.0,"(3600, 22)",79200,"[PAN, PRI, PRD, PT, PVEM, MC]",[C. Independiente 2],3,{int64},Only One Type
16,16_michoacan,16_michoacan/16_michoacan_casillas/michoacan_municipios_2012_polling_place (1),1,24,112,1.0,113.0,"(6026, 23)",138598,"[PAN, PRI, PRD, PT, PVEM, MC]",e,0,{int64},Only One Type
25,20_oaxaca,20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place,416,"3, 4, 9, 10, 13, 14, 18,",153,2.0,570.0,"(3164, 20)",63280,"[PAN, PRI, PRD, PVEM, PT, MC]",e,0,"{object, int64}","{'section_id': int64, 'local_district_id': int64, 'municipality_id': int64, 'state_id': int64, 'PAN': int64, 'PRI': int64, 'PRD': int64, 'PVEM': int64, 'PT': int64, 'MC': object, 'PUP': object, 'PANAL': object, 'PSD': object, 'PAN,PRD,PT': int64, 'PAN,PRD': int64, 'PAN,PT': int64, 'PRD,PT': int64, 'PRI,PVEM': int64, 'Not_reg': int64, 'Null_votes': int64}"
26,20_oaxaca,20_oaxaca/oaxaca_casillas/oaxaca_municipios_2016_polling_place,416,"3, 4, 9, 10, 13, 14, 18,",153,2.0,570.0,"(3314, 26)",86164,"[PAN, PRI, PRD, PVEM, PT, MC]",[C. Independiente 4],5,"{int64, float64}","{'section_id': int64, 'local_district_id': int64, 'municipality_id': int64, 'state_id': int64, 'PAN': int64, 'PRI': int64, 'PRD': int64, 'PVEM': int64, 'PT': int64, 'MC': int64, 'PUP': int64, 'PANAL': int64, 'PSD': float64, 'MORENA': int64, 'PES': int64, 'PRS': int64, 'PAN,PRD': int64, 'PRI,PVEM': int64, 'PANAL,PSD': int64, 'C. Independiente 1': int64, 'C. Independiente 2': int64, 'C. Independiente 3': int64, 'C. Independiente 4': int64, 'C. Independiente 5': int64, 'Not_reg': int64, 'Null_votes': int64}"
27,20_oaxaca,20_oaxaca/oaxaca_casillas/oaxaca_municipios_2018_polling_place,416,"3, 4, 9, 10, 13, 14, 18,",153,2.0,570.0,"(3458, 69)",238602,"[PAN, PRI, PRD, PVEM, PT, MC]",[C. Independiente 22],23,{int64},Only One Type
30,21_puebla,21_puebla/puebla_casillas/puebla_municipios_2018_polling_place,1,174,216,1.0,217.0,"(7309, 54)",394686,"[PAN, PRI, PRD, PT, PVEM, MC]",[C. Independiente 1],2,{int64},Only One Type
37,24_san-luis-potosi,24_san-luis-potosi/san-luis-potosi_casillas/san-luis-potosi_municipios_2015_polling_place,2,"29, 50",56,1.0,58.0,"(3397, 44)",149468,"[PAN, MC, PRD, PT, PANAL, PRI]",[],1,{int64},Only One Type
38,24_san-luis-potosi,24_san-luis-potosi/san-luis-potosi_casillas/san-luis-potosi_municipios_2018_polling_place,1,nan,59,1.0,58.0,"(3625, 44)",159500,"[PAN, PRI, PRD, PT, PMC, PVEM]",[C. Independiente 11],12,"{int64, float64}","{'section_id': float64, 'local_district_id': float64, 'municipality_id': float64, 'state_id': float64, 'PAN': float64, 'PRI': float64, 'PRD': float64, 'PT': float64, 'PMC': float64, 'PVEM': float64, 'PCP': float64, 'PANAL': float64, 'MORENA': float64, 'PES': float64, 'PT,MORENA': float64, 'PAN,PMC': float64, 'PRI,PVEM': float64, 'PRI,PANAL': float64, 'PRI,PCP': float64, 'PT,PES': float64, 'MORENA,PES': float64, 'PAN,PRD': float64, 'PRD,PMC': float64, 'PVEM,PANAL': float64, 'PRI,PVEM,PCP': float64, 'PAN,PRD,PMC': float64, 'PRI,PVEM,PANAL': float64, 'PRI,PANAL,PCP': float64, 'PT,MORENA,PES': float64, 'PRI,PVEM,PANAL,PCP': float64, 'C. Independiente 1': float64, 'C. Independiente 2': float64, 'C. Independiente 3': float64, 'C. Independiente 4': float64, 'C. Independiente 5': float64, 'C. Independiente 6': float64, 'C. Independiente 7': float64, 'C. Independiente 8': float64, 'C. Independiente 9': float64, 'C. Independiente 10': float64, 'C. Independiente 11': float64, 'C. Independiente 12': float64, 'Not_reg': float64, 'Null_votes': int64}"
44,27_tabasco,27_tabasco/tabasco_casillas/tabasco_municipios_2015_polling_place,1,4,16,1.0,17.0,"(1958, 22)",43076,"[PAN, PRI, PRD, PVEM, PT, MC]",[C. Independiente 4],5,{int64},Only One Type
48,28_tamaulipas,28_tamaulipas/tamaulipas

['13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2016_polling_place', '16_michoacan/16_michoacan_casillas/michoacan_municipios_2012_polling_place (1)', '20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place', '20_oaxaca/oaxaca_casillas/oaxaca_municipios_2016_polling_place', '20_oaxaca/oaxaca_casillas/oaxaca_municipios_2018_polling_place', '21_puebla/puebla_casillas/puebla_municipios_2018_polling_place', '24_san-luis-potosi/san-luis-potosi_casillas/san-luis-potosi_municipios_2015_polling_place', '24_san-luis-potosi/san-luis-potosi_casillas/san-luis-potosi_municipios_2018_polling_place', '27_tabasco/tabasco_casillas/tabasco_municipios_2015_polling_place', '28_tamaulipas/tamaulipas_casillas/tamaulipas_municipios_2018_polling_place', '7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place']


In [166]:
#@title Ver Total Ids únicos = -1
df_continuity=df.copy()
filter_df_continuity=df_continuity[ df_continuity['Total Ids únicos']==-1]
estados=list(map(lambda x: x.split('/')[0], filter_df_continuity['Libro']))
filter_df_continuity.insert(0, "Estado", estados )
#filter_df_continuity.to_excel('/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/data_from_notebook/temp_file.xlsx')
display(HTML(filter_df_continuity.to_html()))

,Estado,Libro,Total_No_continuos,Lista IDs NO Continuos,Total Ids únicos,Mínimo,Máximo,"Filas,Columnas",Total Celdas,columns,filter_independents,total_independents,types,temp_types_by_column
8,13_hidalgo,13_hidalgo/13_hidalgo_casillas/hidalgo_municipios_2017_polling_place,0,,-1,45.0,45.0,"(14, 11)",154,"[section_id, local_disctrict_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, Null_votes]",Sin independientes,0,{int64},Only One Type
76,9_ciudad-de-mexico,9_ciudad-de-mexico/ciudad-de-mexico_casillas/ciudad-de-mexico_municipios_2018_polling_place,0,,-1,0.0,0.0,"(12975, 29)",376275,"[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, PANAL, MORENA, PES, PH, PT,MORENA,PES, PT,MORENA, PT,PES, MORENA,PES, PAN,PRD,MC, PAN,PRD, PAN,MC, PRD,MC, C. Independiente 1, C. Independiente 2, C. Independiente 3, C. Independiente 4, C. Independiente 5, Not_reg, Null_votes]","[C. Independiente 1, C. Independiente 2, C. Independiente 3, C. Independiente 4, C. Independiente 5]",5,{int64},Only One Type


---------------------

#SECTION 2: SABANA

* Crea hojas nuevas

In [168]:
mi_lista=['20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place',
          '7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place']

In [169]:
import pandas as pd
from IPython.display import display, HTML
from openpyxl import load_workbook

#state_origin =  1#@param {type:"integer"}
scan_sheet =  'Polling_places' #@param {type:"raw"}
sheet=scan_sheet
new_sheet =  'Polling_places_sabana_sergs' #@param {type:"raw"}
column_origin ='section_id'  #@param {type:"string"}
new_column ='new_municipality_ids'  #@param {type:"string"}
name_scan_column=  'municipality_id' #@param {type:"string"}


content_books=mi_lista  #@param {type:"raw"}
origin_computes='/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/'
content_books=list(map(lambda x: origin_computes+x+'.xlsx' ,content_books))

#content_books=['/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx'] #@param {type:"string"}

sabana_dir = "/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/respaldos/SabanaDatos_updated.xlsx"
column_find='IDEDO_IDSECC'

#dir_content_books = "/gdrive/My\ Drive/PRI_CNIP_ESTRUCTURAS/computos/1_aguascalientes/aguascalientes_casillas/"#@param {type:"raw"}
#patron_list_files="*mun*2013*xlsx| tr '\n' '\n'" #@param {type:"raw"}
#content_books=!ls {dir_content_books+patron_list_files }


def scan_column_zero(name_scan_column,df):
  '''Return true if name_scan_column sum zero for a dataframe '''
  sum=df[name_scan_column].sum()
  if sum == 0:
    print(f'La columna de {name_scan_column} suma cero')
    return True
  return False
    
def add_new_sheet(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists it will be overwritten'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets) 
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

def mun_id(column_find,section_id, state_id,df_sabana):
  ''' find mun id in column concat state+_+section_id if not found it return 0'''
  keys = str(state_id) + '_' + str(section_id)
  mun = list(df_sabana[df_sabana[column_find]==keys]['IDMPIO'].values)
  if mun:
    return mun[0]
  else:
    return 0

def sabana(column_origin, new_column, sheet, content_books, new_sheet, name_scan_column):
  '''create a new sheet o replace with new columns with id's from sabana file '''
  dicc_info={}
  list_dicc_info=[]
  muns_id=[]
  zero_sections_muns=[]
  df_sabana=pd.read_excel(sabana_dir)
  preview_mun=False
  for book in content_books:
    state_origin=int(book.split(origin_computes)[1].split('_')[0])
    #print(state_origin)
    if state_origin == 16: # error in this files :(
      continue
    preview_mun=False

    print(book)
    try:
      _df_origin = pd.read_excel(book, sheet_name=sheet)
    except:
      print('hey can\'t read this file !')
      continue
    
    if  not scan_column_zero(name_scan_column,_df_origin):
      #print(f'La columna de {name_scan_column} ya tiene valores previos')
      preview_mun=True

    list_b = list(_df_origin[column_origin])
    
    for section in list_b:
      _mun_id = mun_id(column_find=column_find, 
                       section_id=section, 
                       state_id=state_origin, 
                       df_sabana=df_sabana)
      
      muns_id.append(_mun_id)
      if _mun_id == 0:
        zero_sections_muns.append(section)

    #_df_origin[new_column]=muns_id
    _df_origin.insert(4, new_column,muns_id  )
    comparation=list(map(lambda x,y: x==y, _df_origin[name_scan_column], _df_origin[new_column]))
    _df_origin.insert(5, 'comparation',comparation)

    #display(HTML(_df_origin.head(1).to_html()))

    add_new_sheet(name_book=book,df=_df_origin,name_sheet=new_sheet)  
    '''print('\nSecciones que no tuvieron conversión a su id municipality y quedaron en ceros: ')
    print(set(zero_sections_muns))
    print('\nSummary comparation')
    print(set(comparation))
    print('\nPreview municipality')
    print(preview_mun)'''


    dicc_info[book]={'book': book,'preview_mun': preview_mun, 'zero_sections': set(zero_sections_muns), 'comparation': set(_df_origin['comparation'].values)}
    list_dicc_info.append(dicc_info[book])
    zero_sections_muns=[]
    muns_id=[]
  return list_dicc_info


list_dicc_info=sabana(column_origin, new_column, sheet, content_books, new_sheet, name_scan_column)





df_info=pd.DataFrame(list_dicc_info)
display(HTML(df_info.to_html()))

  

/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx
/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx


,book,preview_mun,zero_sections,comparation
0,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx,True,{},"{False, True}"
1,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx,True,{},{True}


# SECTION 3: ANALYZE EMPTY CELLS


* No crea hojas nuevas

Esta sección nos ayuda a identificar las columnas que tienen celdas con NaN("Not a Number" para nuestro caso igual a:  "vacías"), así como el tipo de dato presente en cada columna.

> * Recibe una lista con los nombres de las hojas que deseamos leer en cada libro.
> * También recibe una cadena que representa un patrón, el cuál busca los archivos excel cargados.
> * A veces solo nos interesa ver la info de las hojas que tienen celdas NaN, entonces hay que marcar la casilla filter_only_sheets_nan.
> * Marca la casilla print_list_sheets si quieres ver el contenido de cada hoja procesada.

In [170]:
mi_lista=['20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place',
          '7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place']

In [171]:
#@title Summary NaN types

import pandas as pd
from IPython.display import display, HTML

sheets =  ['Election','Polling_places']#@param {type:"raw"}
filter_only_sheets_nan = False #@param {type:"boolean"}
view_list_sheets = False #@param {type:"boolean"}

#content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
#content_books=!ls {content_books}


content_books=mi_lista  #@param {type:"raw"}
origin_computes='/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/'
content_books=list(map(lambda x: origin_computes+x+'.xlsx' ,content_books))



def count_nan_in_columns_by_sheet(content_books, sheets):
  '''Return dicc with conteo_NaN info by sheet in content_books ''' 
  list_data=[]
  dicc_data={}

  for sheet in sheets:
    for file_name in content_books:
      try:
        sheet_to_df = pd.read_excel(file_name, sheet_name=sheet)
      except:
        print(f'\n ¡AVISO! No existe la hoja {sheet} en este archivo {file_name}')
        continue

      shape = sheet_to_df.shape
      tipos=set(sheet_to_df.dtypes.values)
      

      df_types = pd.DataFrame([sheet_to_df.dtypes], columns=sheet_to_df.columns)

      df_types = df_types.append(sheet_to_df, ignore_index=True)
      df_types = df_types.rename(index = {0: 'data_column_type'})
      
      dicc_data = {'Archivo': file_name}
      dicc_data.update({'Hoja': sheet})
      dicc_data.update({'Contenido': str(shape[0]) + ' filas, '+ str(shape[1])+ ' columnas'})

      if sheet_to_df.isnull().sum().sum() == 0:
          msg = 'No tiene ninguna celda tipo NaN'
          
      else:
          null_columns = sheet_to_df.columns[sheet_to_df.isnull().any()]
          msg=''.join(sheet_to_df[null_columns].isnull().sum().to_string()).split('\n')
          msg=', '.join(msg)

      if len(tipos)>1:
            msg = msg + ', pero tiene más de un tipo de datos' + str(tipos)

      dicc_data.update({'Aviso': msg })
      list_data.append(dicc_data)

      if view_list_sheets:

          print('\nArchivo:',file_name)
          print('________                                                    ')
          print('\t| Hoja:',sheet, '\t',shape[0], 'filas, ', shape[1], 'columnas')
          print('\t| Aviso conteo NaN:',msg,' |\n')
          if 'Polling_places' in sheet:
            display(HTML(df_types.head(2).to_html()))
          else:
            display(HTML(df_types.to_html()))



  return list_data


list_data=count_nan_in_columns_by_sheet(content_books, sheets)

df=pd.DataFrame([[fila['Archivo'], 
                  fila['Aviso'], 
                  fila['Contenido'], 
                  fila['Hoja']] 
                 for fila in list_data] , columns=['archivo', 'conteo_NaN', 'contenido', 'hoja'])

if filter_only_sheets_nan:
  df=df[~df.conteo_NaN.str.contains('No tiene')]

print('\nRESUMEN:\n')
display(HTML(df.to_html()))


RESUMEN:



,archivo,conteo_NaN,contenido,hoja
0,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx,"No tiene ninguna celda tipo NaN, pero tiene más de un tipo de datos{dtype('O'), dtype('int64')}","1 filas, 7 columnas",Election
1,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx,"No tiene ninguna celda tipo NaN, pero tiene más de un tipo de datos{dtype('O'), dtype('int64')}","1 filas, 7 columnas",Election
2,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx,"No tiene ninguna celda tipo NaN, pero tiene más de un tipo de datos{dtype('O'), dtype('int64')}","3164 filas, 20 columnas",Polling_places
3,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx,No tiene ninguna celda tipo NaN,"6303 filas, 77 columnas",Polling_places


# SECTION 4: CONVERSION TO INTEGERS
######  
* Si crea hojas nuevas

> *   Modifica a integer los valores de las columnas
*   De no ser posible la conversión, intenta reemplazar a 0's espacios en blanco y los que cumplan con un patrón regular.
*   En el mismo archivo agrega una hoja nueva 'Polling_places_corrections'

In [172]:
#@title Definir funciones a usar
import pandas as pd
import numpy as np
from openpyxl import load_workbook

def add_new_sheet_index(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists new sheet with other index will be created'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

def add_new_sheet(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists it will be overwritten'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets) 
  #print(writer.sheets) 
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()

def types_columns_unique(types_dataframe):
  ''' generate a set of types columns from columns dataframe'''
  return ','.join(map(str, set(list(types_dataframe))))


def adjust_integers(content_books, sheet_name, new_sheet_name, regex_pattern_aditional=r'^-*$|^o+$|^O+$'):
  ''' Try to convert all columns in a sheet_name to integer, 
      from each book in content_books '''
  all_columns_integer={}
  print(f'\nSe creará NUEVA HOJA "{new_sheet_name}" para cada uno de los archivos siguientes, con los ajustes que se muestran en summary')
  for book in content_books:
    sustituir_vacios = True
    original_all_numbers = False
    print('\n\nLeyendo Libro:',book)
    
    try:
      df_temp = pd.read_excel(book,sheet_name=sheet_name)
    except:
      print(f'\tNo tiene la hoja: {sheet_name}')
      continue

    all_columns_integer[book]={}

    try:
      df_temp = df_temp.astype(int, errors='raise')
      sustituir_vacios = False
      original_all_numbers = True
    except ValueError as e:
      print('\n\tAVISO: Se encontraron valores que no se pueden convertir a entero directamente...')
      #print('\n\t\t!Info error 1:',e)
      all_columns_integer[book]['integer'] = False
    if sustituir_vacios:
      try:
        df_temp=df_temp.replace(r'^\s*$', 0, regex=True).replace(regex_pattern_aditional, 0, regex=True)
        #print(f'\n\tTRATAREMOS de SUSTITUIR valores VACÍOS, y los que cumplan este PATRÓN: {regex_pattern_aditional} por ceros')
      except ValueError as e:
        print('\n\tAVISO: Algún valor NO SE PUDO SUSTITUIR por 0\'s \:\( ')
        #print ('\n\t\t!Info error 2:',e)
    df_temp=df_temp.fillna(0)

    try:
      df_temp = df_temp.astype(int, errors='raise')
      all_columns_integer[book]['integer'] = True
    except ValueError as e:
      print('\n\tAVISO: Existe algún valor que no se puede convertir a entero :S')
      #print('\n\t\t!Info error 3', e)
      all_columns_integer[book]['integer'] = False
    
    all_columns_integer[book]['original_all_numbers'] = original_all_numbers
    all_columns_integer[book]['types_unique'] = types_columns_unique(df_temp.dtypes)

    g = dict(df_temp.columns.to_series().groupby(df_temp.dtypes).groups)   
    types_columns={}

    for k,v in g.items():
      if not 'int' in (str(k)):
        types_columns[str(k)]=','.join(list(v))

    if not all_columns_integer[book]['integer']:
      print('\n\tAVISO: Intenta un proceso individual para cada columna ...')
    types_columns.update({'book':book.split('.')[0]})
    all_columns_integer[book]['types_by_column'] = types_columns
    add_new_sheet(name_book=book,df=df_temp,name_sheet=new_sheet_name)


  print('\nFin del proceso :)')
  return all_columns_integer


In [173]:
mi_lista=['20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place',
          '7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place']

In [174]:
#@title Column to Integer Conversion Proccess

content_books=mi_lista  #@param {type:"raw"}
origin_computes='/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/'
content_books=list(map(lambda x: origin_computes+x+'.xlsx' ,content_books))

all_columns_integer = adjust_integers(content_books = content_books, 
                                    sheet_name = 'Polling_places', 
                                    new_sheet_name = 'Polling_places_corrections',
                                    regex_pattern_aditional = r'^-*$|^o+$|^O+$|^/+$')

print('Summary_integers')
df_sumary_integers = pd.DataFrame([[key.split('.')[0],value['integer'],value['original_all_numbers'], value['types_unique']] for key,value in all_columns_integer.items()], columns=['name_book', 'Finally Integer?','Began with numbers?', 'Types unique'])
display(HTML(df_sumary_integers.to_html()))


Se creará NUEVA HOJA "Polling_places_corrections" para cada uno de los archivos siguientes, con los ajustes que se muestran en summary


Leyendo Libro: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx

	AVISO: Se encontraron valores que no se pueden convertir a entero directamente...

	AVISO: Existe algún valor que no se puede convertir a entero :S

	AVISO: Intenta un proceso individual para cada columna ...


Leyendo Libro: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx

Fin del proceso :)
Summary_integers


,name_book,Finally Integer?,Began with numbers?,Types unique
0,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place,False,False,"object,int64"
1,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place,True,True,int64


## SUBSECTION 2.1 Suggest changes in columns cells

In [175]:
#@title Summary Columns cannot convert to Integers in a book
df_columnas_not_int = pd.DataFrame([v['types_by_column'] for v in all_columns_integer.values() if not v['integer']])
display(HTML(df_columnas_not_int.to_html()))


,object,book
0,"MC,PUP,PANAL,PSD",/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place


In [176]:
#@title Proccess to suggest changes in columns
def fix_multiple_types(n): 
    import re
    tipo=str(type(n))
    x=n
    if 'int' in tipo:
      return n
    if 'str' in tipo:
      numbers_patron = re.compile(r"[\d]", re.X)
      number=''.join(numbers_patron.findall(n))
      try: 
        n=int(number)
        print( '\t\t\tAhora: ',str(type(n)), str(n), '\tAntes:', tipo, str(x))
        return n
      except:
        pass
    elif 'float' in tipo:
      if n >= 0:
        n=int(n)
        return n
    print('\t\t\tAhora:', str(type(0)),str(0), '\tAntes:', str(type(x)), str(x))
    return 0

def fix_multiple_types_suggest_columns(n): 
    import re
    tipo=str(type(n))
    x=n
    if 'int' in tipo:
      return n
    if 'str' in tipo:
      numbers_patron = re.compile(r"[\d]", re.X)
      number=''.join(numbers_patron.findall(n))
      try: 
        n=int(number)
        n=''.join(['Ahora: ',str(type(n)),' ', str(n), ' Antes: ', tipo, str(x)])
        return n
      except:
        pass
    elif 'float' in tipo:
      if n >= 0:
        n=int(n)
        return n
    return ''.join(['Ahora: ', str(type(0)),' ', str(0), ' Antes: ', str(type(x)), str(x)])
   

for k,v in all_columns_integer.items():
  df_temp_columns_suggest = pd.read_excel(k,sheet_name='Polling_places')
  df_temp_sheet_suggest = pd.read_excel(k,sheet_name='Polling_places')

  print('LIBRO:',k,'\t')
 
  if not v['integer']:
    for key, tipos in v['types_by_column'].items():
      if key == 'book':
        continue
      print('TIPO:',key)
      
      for c in tipos.split(','):
        print('\n\tColumna:\n\t',c,'\n')
        df_temp_sheet_suggest[c]=list(map(fix_multiple_types,list(df_temp_sheet_suggest[c])))
        #df_temp_columns_suggest[c+'suggested']= df_temp_columns_suggest[c].map(fix_multiple_types_suggest_columns)
        df_temp_columns_suggest[c]= df_temp_columns_suggest[c].map(fix_multiple_types_suggest_columns)

    df_temp_sheet_suggest.astype(int, errors='ignore')
    df_temp_columns_suggest.astype(int, errors='ignore')
    

    add_new_sheet(name_book=k,df=df_temp_sheet_suggest,name_sheet='Polling_places_corrections')
    add_new_sheet(name_book=k,df=df_temp_columns_suggest,name_sheet='Polling_places_suggested')


LIBRO: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx 	
TIPO: object

	Columna:
	 MC 

			Ahora: <class 'int'> 0 	Antes: <class 'str'> P.N
			Ahora: <class 'int'> 0 	Antes: <class 'str'> P.N
			Ahora: <class 'int'> 0 	Antes: <class 'str'> P.N
			Ahora: <class 'int'> 0 	Antes: <class 'str'> P.N

	Columna:
	 PUP 

			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P

	Columna:
	 PANAL 

			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0 	Antes: <class 'str'> N.P
			Ahora: <class 'int'> 0

# SECTION 5: ANALYZE 'Election'

In [177]:
mi_lista=['20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place',
          '7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place']

In [178]:
#@title Comprobar hoja Election
columnas_plantilla_election1=['State','Year','Election_type','Total_votes','Nominal_list','Computed_transactions','Fidelity']
#columnas_plantilla_Polling_places=['section_id', 'local_disctrict_id', 'municipality_id', 'state_id','Not_reg','Null_votes']
columnas_plantilla_election=set(['State','Year','Election_type','Total_votes','Nominal_list','Computed_transactions','Fidelity'])
lista_elections=[]
dicc_election={}

from IPython.display import display, HTML
import pandas as pd

#content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
#content_books =  !ls {content_books}


content_books=mi_lista  #@param {type:"raw"}
origin_computes='/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/'
content_books=list(map(lambda x: origin_computes+x+'.xlsx' ,content_books))


for k in content_books:
  dicc_election={}
  print('|',k)
  try:
    xls = pd.ExcelFile(k)
  except:
    print('No es un archivo válido')
    continue
  if not 'Election' in xls.sheet_names:
    print(f'\n\tNo existe la hoja Election para este archivo {k}')
    continue
  try:
    data_election=pd.read_excel(k,sheet_name='Election')
  except:
    print(f'No se puede leer el archivo{k}')
    continue
  dicc_election['book']=k
  columnas_hoja_election=set(data_election.columns)

  if  not columnas_plantilla_election-columnas_hoja_election:
    #print('\tColumnas igual a la plantilla!\n')
    header_equal=True
  else:
    print('AVISO! Falta al menos una columna con respecto a la plantilla\n')
    header_equal=False

  dicc_election['header_equal']=header_equal

  dicc_election['types_column']=list(data_election.dtypes)
  try:
    dicc_election['values_fila']=list(data_election.iloc[0])
  except:
    dicc_election['values_fila']=list(['']*len(columnas_plantilla_election))
    print('AVISO! Fila vacía')

  lista_elections.append(dicc_election)
  df_=pd.DataFrame([dicc_election['types_column'],dicc_election['values_fila'] ], index=['types','values'],columns=columnas_plantilla_election1)
  print()
  display(HTML(df_.to_html()))
  print()

df_2=pd.DataFrame(lista_elections)
display(HTML(df_2.to_html()))





| /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx



,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
types,int64,int64,object,int64,int64,int64,int64
values,20,2013,Municipios,1034524,2669909,0,3



| /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx



,State,Year,Election_type,Total_votes,Nominal_list,Computed_transactions,Fidelity
types,int64,int64,object,int64,int64,int64,int64
values,7,2018,Municipios,2365367,3541181,0,3


,book,header_equal,types_column,values_fila
0,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx,True,"[int64, int64, object, int64, int64, int64, int64]","[20, 2013, Municipios, 1034524, 2669909, 0, 3]"
1,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx,True,"[int64, int64, object, int64, int64, int64, int64]","[7, 2018, Municipios, 2365367, 3541181, 0, 3]"


# SECTION 6: EDITING COLUMN NAMES

In [179]:
#@title Definir funciones a usar
def reemplazar_nombres_columnas(df_temp, reemplazar):
  '''Recibe un nuevo nombre de columna y lo reemplaza por una anteriormente dado'''
  
  if reemplazar == 'Y':

    print("\t¿Cuál es nombre de columna a reemplazar? ", end="")
    columna_anterior = input()
    print("\t¿Cuál es el nuevo nombre de columna?", end="")
    columna_nueva = input()
    print(f"\n\tSe reemplazará: {columna_anterior} por {columna_nueva}")

    if columna_anterior in df_temp.columns and columna_nueva:
  
      df_temp.rename(columns={ columna_anterior :columna_nueva},inplace=True)
      
      print ('\n\tREEMPLAZO LISTO!\n')
      print('\n\tESTAS SON LAS COLUMNAS AHORA:\n\n\t',df_temp.columns)
    else:
      print(f'{columna_anterior} No es una columna existente! y/o recuerda agregar el texto por el cual sustituir')

  print('\n\t ¿Deseas reemplazar el nombre de otra columna? (Y/N)')
  reemplazar = input()

  if reemplazar == 'Y':
    df_temp=reemplazar_nombres_columnas(df_temp, reemplazar)
    print('\tSe actualizará el nombre de la columna en el archivo de excel')
  else:
    print('\tFinalizando edición de columnas de este archivo...')

  return df_temp


def rename_columns(content_books=[], sheet_name='Polling_places_corrections'):
  '''Reads the name and renames certain columns of a sheet'''
  for book in content_books:

    print('\n\nNOMBRE DEL ARCHIVO:',book)
    try:
      df_temp=pd.read_excel(book,sheet_name=sheet_name)
    except:
      print(f'No existe la hoja {sheet_name} en el archivo: {book}')
      continue
    print('\n\n\tESTAS SON LAS COLUMNAS PRESENTES:\n\n')
    print('\t',df_temp.columns)
    print('\n\t¿Deseas reemplazar el nombre de alguna columna? (Y/N)')

    reemplazar = input()

    if reemplazar == 'Y': 
      df_temp = reemplazar_nombres_columnas(df_temp,reemplazar)
      add_new_sheet(name_book=book,df=df_temp,name_sheet=sheet_name)
      print('\tSe actualizó el nombre de la columna en el archivo de excel')
    else:
      if reemplazar == 'N':
        print('\tBye!')
      else:
        print('\tOpción no válida, pasaremos al siguiente archivo...')

  return print('\nProceso finalizado :D')

In [180]:
mi_lista=['20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place',
          '7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place']

In [181]:
#@title Proccess to help us to rename columns

from IPython.display import display, HTML


#content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
#content_books =  !ls {content_books}

content_books=mi_lista  #@param {type:"raw"}
origin_computes='/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/'
content_books=list(map(lambda x: origin_computes+x+'.xlsx' ,content_books))


sheet_name="Polling_places_sabana_sergs"#@param {type:"string"}


rename_columns(content_books=content_books, sheet_name=sheet_name)



NOMBRE DEL ARCHIVO: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx


	ESTAS SON LAS COLUMNAS PRESENTES:


	 Index(['section_id', 'local_district_id', 'municipality_id', 'state_id',
       'new_municipality_ids', 'comparation', 'PAN', 'PRI', 'PRD', 'PVEM',
       'PT', 'MC', 'PUP', 'PANAL', 'PSD', 'PAN,PRD,PT', 'PAN,PRD', 'PAN,PT',
       'PRD,PT', 'PRI,PVEM', 'Not_reg', 'Null_votes'],
      dtype='object')

	¿Deseas reemplazar el nombre de alguna columna? (Y/N)
N
	Bye!


NOMBRE DEL ARCHIVO: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx


	ESTAS SON LAS COLUMNAS PRESENTES:


	 Index(['section_id', 'local_district_id', 'municipality_id', 'state_id',
       'new_municipality_ids', 'comparation', 'PAN', 'PRI', 'PRD', 'PT',
       'PVEM', 'MC', 'PANAL', 'PCU', 'MORENA', 'PES', 'MVC', 'PAN,PRD,MC',
       'PAN,PRD', 'PAN,MC', 'PRD,MC', 'MORENA,PT,PES', 

# SECCIÓN 7:  Split Computations

* Crea la hoja Polling_places_split

Esta sección nos ayuda a separar una columna apilada dada (C. Independiente n), en múltiples columnas agrupando por una columna pivote (municipality_id)

> * Agregar el nombre de la hoja a leer,
> * Agregar el nombre de la nueva hoja a crear
> * Agregar el patrón de los archivos a leer.




In [ ]:
#@title Functions [Split Computations]
import pandas as pd
from IPython.display import display, HTML
from openpyxl import load_workbook


def some_column_begin_with(columns, phrase='C. Independiente'):
  for column in columns:
    if phrase in column:
      return True
  return False

def filter_columns(columns, phrase='C. Independiente'):
    return filter(lambda c: phrase in c, columns)

def add_new_sheet(name_book,df,name_sheet):
  ''' add new sheet, if the sheet exists it will be overwritten'''
  book = load_workbook(name_book)
  writer = pd.ExcelWriter(name_book, engine = 'openpyxl', mode='w')
  writer.book = book
  writer.sheets = dict((ws.title, ws) for ws in book.worksheets) 
  df.to_excel(writer, sheet_name = name_sheet, index=False)
  writer.save()
  writer.close()


def split_column_in_columns(content_books,sheet, sort_column):
  '''split a stacked column
  (Independent C. n's), in multiple columns
  by filter value of another column (municipality_id)'''

  new_computes=[]
  computes_mun=[]
  phrase='C. Independiente' 
  pivot_column='municipality_id'
  report_sheet_created={}
  list_report_sheet_created=[]

  for file_name in content_books:
    new_computes=[]
    try:
      my_data = pd.read_excel(file_name, sheet_name=sheet)
    except:
      print(f'No existe Polling places en el archivo o no se puede acceder a él {file_name}')
      continue
    
    my_data = my_data.sort_values(sort_column)

    if some_column_begin_with(my_data.columns, phrase):
      muns = my_data[pivot_column].unique()

      #print(muns)
      for mun in muns:
        df_mun = my_data.loc[my_data[pivot_column] == mun].copy()

        for n_independiente in filter_columns(my_data.columns, phrase):
          if df_mun[n_independiente].sum() > 0:
            df_mun[n_independiente +'_mun_'+str(mun)] = df_mun[n_independiente]
            
        computes_mun = df_mun.to_dict('records')
        new_computes.extend(computes_mun)
         
      new_data = pd.DataFrame(new_computes)
      new_data = new_data.fillna(0)
      new_data = new_data.astype(int)

      print(f'Se creó la hoja: {new_sheet} en el archivo:{file_name}')
      add_new_sheet(file_name, new_data, new_sheet)
      sheet_created=True
      report_sheet_created={
          'file_name': file_name,
          'sheet_created': sheet_created,
          'new_data_columns': new_data.columns.values,
          'new_data_shape': new_data.shape}

    else:
      print(f'No existen C. Independientes en el archivo: {file_name},solo se ordenó por {sort_column}')
      add_new_sheet(file_name, my_data, sheet)
      sheet_created=False
      report_sheet_created={
          'file_name': file_name,
          'sheet_created': sheet_created,
          'new_data_columns': my_data.columns.values,
          'new_data_shape': my_data.shape}
    list_report_sheet_created.append(report_sheet_created)
    computes_mun={}
  return list_report_sheet_created


In [ ]:
mi_lista=['8_chihuahua/8_chihuahua_casillas/chihuahua_municipios_2016_polling_place', 
          '20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place',
          '7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place']

In [182]:
#@title Split columns from computations

sheet="Polling_places"#@param {type:"string"}
new_sheet="Polling_places_split"#@param {type:"string"}
sort_column="municipality_id"#@param {type:"string"}


#content_books =  "*.xlsx | tr '\n' '\n'"#@param {type:"raw"}
#content_books=!ls {content_books}


content_books=mi_lista  #@param {type:"raw"}
origin_computes='/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/'
content_books=list(map(lambda x: origin_computes+x+'.xlsx' ,content_books))






list_report_sheet_created=split_column_in_columns(content_books,sheet, sort_column)


df_split=pd.DataFrame([[fila['file_name'].split('.')[0], 
                  fila['sheet_created'], 
                  fila['new_data_shape'], 
                  fila['new_data_columns']] 
                 for fila in list_report_sheet_created] , columns=['archivo', 'hoja Nueva?', 'filas,columnas', 'name_columns'])


print('\nRESUMEN:\n')
display(HTML(df_split.to_html()))


No existen C. Independientes en el archivo: /gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place.xlsx,solo se ordenó por municipality_id
Se creó la hoja: Polling_places_split en el archivo:/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place.xlsx

RESUMEN:



,archivo,hoja Nueva?,"filas,columnas",name_columns
0,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/20_oaxaca/oaxaca_casillas/oaxaca_municipios_2013_polling_place,False,"(3164, 20)","[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PVEM, PT, MC, PUP, PANAL, PSD, PAN,PRD,PT, PAN,PRD, PAN,PT, PRD,PT, PRI,PVEM, Not_reg, Null_votes]"
1,/gdrive/My Drive/PRI_CNIP_ESTRUCTURAS/computos/7_chiapas/chiapas_casillas/chiapas_municipios_2018_polling_place,True,"(6303, 117)","[section_id, local_district_id, municipality_id, state_id, PAN, PRI, PRD, PT, PVEM, MC, PANAL, PCU, MORENA, PES, MVC, PAN,PRD,MC, PAN,PRD, PAN,MC, PRD,MC, MORENA,PT,PES, PT,MORENA, PT,PES, MORENA,PES, PRI,PVEM,PANAL,PCU, PRI,PVEM,PANAL, PRI,PVEM,PCU, PRI,PANAL,PCU, PVEM,PANAL,PCU, PRI,PVEM, PRI,PANAL, PRI,PCU, PVEM,PANAL, PVEM,PCU, PANAL,PCU, C. Independiente 1, C. Independiente 2, C. Independiente 3, C. Independiente 4, C. Independiente 5, C. Independiente 6, C. Independiente 7, C. Independiente 8, C. Independiente 9, C. Independiente 10, C. Independiente 11, C. Independiente 12, C. Independiente 13, C. Independiente 14, C. Independiente 15, C. Independiente 16, C. Independiente 17, C. Independiente 18, C. Independiente 19, C. Independiente 20, C. Independiente 21, C. Independiente 22, C. Independiente 23, C. Independiente 24, C. Independiente 25, C. Independiente 26, C. Independiente 27, C. Independiente 28, C. Independiente 29, C. Independiente 30, C. Independiente 31, C. Independiente 32, C. Independiente 33, C. Independiente 34, C. Independiente 35, C. Independiente 36, C. Independiente 37, C. Independiente 38, C. Independiente 39, C. Independiente 40, C. Independiente 41, Not_reg, Null_votes, C. Independiente 3_mun_2, C. Independiente 4_mun_8, C. Independiente 5_mun_9, C. Independiente 6_mun_9, C. Independiente 7_mun_11, C. Independiente 8_mun_15, C. Independiente 12_mun_17, C. Independiente 13_mun_17, C. Independiente 14_mun_19, C. Independiente 15_mun_21, C. Independiente 9_mun_27, C. Independiente 10_mun_27, C. Independiente 3_mun_29, C. Independiente 17_mun_34, C. Independiente 18_mun_35, C. Independiente 19_mun_46, C. Independiente 20_mun_50, C. Independiente 21_mun_51, C. Independiente 22_mun_59, C. Independiente 23_mun_61, C. Independiente 24_mun_65, C. Independiente 25_mun_65, C. Independiente 26_mun_77, ...]"
